In [ ]:
import pickle
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve

matplotlib.rcParams['mathtext.fontset'] = 'cm'
matplotlib.rcParams.update({'font.size': 15})

### Load data and results

In [ ]:
!ls ../results/

In [ ]:
result_file = "../results/mimic_1_log_p1=6_p2=0.95_p3=0.02.pkl"
dataset = "../data/triage_scenario_1.csv"

In [ ]:
data = pd.read_csv(dataset, index_col = [0, 1])
covariates, target = data.drop(columns = ['D', 'Y1', 'Y2', 'YC', 'acuity', 'nurse']), data[['D', 'Y1', 'YC']]

In [ ]:
results = pickle.load(open(result_file, 'rb'))

### Evaluate 

In [ ]:
def evaluate(results, target, p):
    """
        Evaluate dictionary of results

    Args:
        results (_type_): _description_
        target ()
        p (_type_): _description_

    Returns:
        _type_: Dictionary of results
    """
    evaluation = {}
    
    for i, result in enumerate(results): 
        eval = {
            '$f_Y$': compute_metrics(result['Observed'], target, p),
            '$f_h$': compute_metrics(result['Human'], target, p),
            '$f_\mathcal{A}$': compute_metrics(result['Amalgamation'], target, p),
            '$f_{hyb}$': compute_metrics(result['Hybrid'], target, p),
        }
        try: eval['Observed Negative Outcome'] = {(tar, 'TNR'): (1 - target[tar].loc[results[0].index][covariates.loc[results[0].index].anchor_age == 1]).mean() for tar in target.columns}
        except: pass
        evaluation[i] = pd.DataFrame.from_dict(eval)
    
    evaluation = pd.concat(evaluation)
    evaluation.index.rename(['Fold', 'Outcome', 'Metric'], inplace = True)
    return evaluation 

def compute_metrics(predictions, target, p):
    metrics = {}

    for tar in target.columns:
        metrics[(tar, 'AUC-ROC')] = roc_auc_score(target[tar].loc[predictions.index], predictions)
        try:
            smallest = predictions.nsmallest(n = int(p * len(target)))
            old_test = covariates[covariates.anchor_age == 1].index.intersection(predictions.index)
            oldest   = smallest.loc[smallest.index.intersection(old_test)]
            metrics[(tar, 'TNR')] = len(oldest) / len(old_test)
        except:
            pass

    return metrics

In [ ]:
evaluation_p = {p: evaluate(results, target, p) for p in [0.5]}

### Display

In [ ]:
p = 0.5
metric = 'AUC-ROC'

In [ ]:
colors = ['tab:green', 'tab:red', 'tab:blue', 'tab:orange', 'tab:grey']
patterns = ['/', '-', '\\', '.', '']

In [ ]:
mean = evaluation_p[p].groupby(['Metric', 'Outcome']).mean()
std = evaluation_p[p].groupby(['Metric', 'Outcome']).std()

ax = mean.loc[metric].dropna(axis = 1, how = 'all').plot.bar(edgecolor = 'white', width = 0.8, figsize = (10, 5), yerr = std.loc[metric].dropna(axis = 1, how = 'all'),
                            color = colors)

# Add hatch
hue = mean.loc[metric]
hatches = [p for p in patterns for _ in range(len(hue))]
for i, (bar, hatch) in enumerate(zip(ax.patches, hatches)):
    bar.set_hatch(hatch)

# Add separation lines
lines = np.array([bar.get_x() for bar in ax.patches])
for line in lines[-len(hue):-1] + ((lines[1:len(hue)] - lines[-len(hue):-1] + bar.get_width()) / 2):
    plt.axvline(line, ls = ':', color='grey', linestyle='--')

plt.ylabel(metric)
plt.xticks(rotation = 0)
plt.grid(alpha = 0.5)
plt.legend(bbox_to_anchor=(1.0, 1.0), loc='upper left')
plt.tight_layout()

In [ ]:
all_p = pd.concat(evaluation_p)
all_p.index.rename('Top p %', level = 0, inplace = True)

In [ ]:
mean = all_p.groupby(['Top p %', 'Metric', 'Outcome']).mean()
std = all_p.groupby(['Top p %', 'Metric', 'Outcome']).std()

selection, ax = mean.index.get_level_values('Metric') == metric, None
for model, color in zip(mean.columns, colors): 
    mean_model = mean[model][selection].droplevel('Metric').unstack('Outcome')
    std_model = std[model][selection].droplevel('Metric').unstack('Outcome')
    ax = mean_model.plot(subplots = True, ax = ax, yerr = std_model, layout=(1, len(hue)), legend = False, sharey = True, figsize = (14,5), color = color)

for ax_outcome, outcome in zip(ax, mean_model.columns):
    ax_outcome.set_title(outcome)

ax[0].set_ylabel(metric)
ax[-1].legend(ax[-1].lines, mean.columns, bbox_to_anchor=(1.0, 1.0), loc='upper left')
plt.tight_layout()